In [1]:
import openai
import pinecone 
import tqdm
import requests
import json
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity


/opt/homebrew/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [13]:
API_KEY = "fc24064d3f5746c28e7dbb49a19a2567"
RESOURCE_ENDPOINT = 'https://azure-ozguler-techexplorer.openai.azure.com'

pinecone.init(api_key="6d752c61-d118-49ef-a622-f6f9ff4584f4")
index_name = 'item-embeddings'

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2021-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"
r = requests.get(url, headers={"api-key": API_KEY})
print(r.text)  # see the models deployed in your account 



{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo-ozguler",
      "status": "succeeded",
      "created_at": 1679424805,
      "updated_at": 1679424805,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "text-davinci-003-ozguler",
      "status": "succeeded",
      "created_at": 1679425352,
      "updated_at": 1679425352,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1680255606,
      "updated_at": 1680255606,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scal

In [3]:
### text_columns = ['About Product', 'Product Specification', 'Technical Details', 'Ingredients', 'Direction To Use', 'Product Description']

In [14]:
import pandas as pd
df = pd.read_csv("./netflix_titles.csv")
df = df.loc[:, ["title", "description"]] 
df.shape
df.head()


,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [15]:
import re
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

df["description"] = list(map(lambda x: normalize_text(x), df["description"].values))
df.head()
#print(df["description"].values[1])

,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [16]:
df["embeddings"][1:5] = df["description"].apply(lambda x : get_embedding(x, engine = 'text-search-curie-doc-001'))

RetryError: RetryError[<Future at 0x287cca4d0 state=finished raised InvalidRequestError>]